# Assignment 4
## Understaning scaling of linear algebra operations on Apache Spark using Apache SystemML

In this assignment we want you to understand how to scale linear algebra operations from a single machine to multiple machines, memory and CPU cores using Apache SystemML. Therefore we want you to understand how to migrate from a numpy program to a SystemML DML program. Don't worry. We will give you a lot of hints. Finally, you won't need this knowledge anyways if you are sticking to Keras only, but once you go beyond that point you'll be happy to see what's going on behind the scenes. As usual, we run some import statements:

In [1]:
!pip install --upgrade systemml

Requirement already up-to-date: systemml in /gpfs/global_fs01/sym_shared/YPProdSpark/user/s0f2-ba03446bdf62cc-bd5847e99873/.local/lib/python2.7/site-packages
Requirement already up-to-date: pandas in /gpfs/global_fs01/sym_shared/YPProdSpark/user/s0f2-ba03446bdf62cc-bd5847e99873/.local/lib/python2.7/site-packages (from systemml)
Requirement already up-to-date: scipy>=0.15.1 in /gpfs/global_fs01/sym_shared/YPProdSpark/user/s0f2-ba03446bdf62cc-bd5847e99873/.local/lib/python2.7/site-packages (from systemml)
Requirement already up-to-date: scikit-learn in /gpfs/global_fs01/sym_shared/YPProdSpark/user/s0f2-ba03446bdf62cc-bd5847e99873/.local/lib/python2.7/site-packages (from systemml)
Requirement already up-to-date: Pillow>=2.0.0 in /gpfs/global_fs01/sym_shared/YPProdSpark/user/s0f2-ba03446bdf62cc-bd5847e99873/.local/lib/python2.7/site-packages (from systemml)
Requirement already up-to-date: numpy>=1.8.2 in /gpfs/global_fs01/sym_shared/YPProdSpark/user/s0f2-ba03446bdf62cc-bd5847e99873/.local/

In [ ]:
'''
import pip
 
try:
    __import__('pandas')
except ImportError:
    pip.main(['install', 'pandas']) 
    
try:
    __import__('dateutil')
except ImportError:
    pip.main(['install', 'dateutil'])     
    
try:
    __import__('systemml')
except ImportError:
    pip.main(['install', 'systemml'])         
'''

In [ ]:
#!pip uninstall python-dateutil
#!pip install python-dateutil --upgrade
'''
pip.main(['uninstall', 'python-dateutil']) 
pip.main(['install', 'python-dateutil']) 
'''

In [ ]:
'''
from pandas.compat.numpy import dateutil
'''

In [2]:
from systemml import MLContext, dml
import numpy as np
import time

/gpfs/fs01/user/s0f2-ba03446bdf62cc-bd5847e99873/.local/lib/python2.7/site-packages/pandas/_libs/__init__.py:4: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
/gpfs/fs01/user/s0f2-ba03446bdf62cc-bd5847e99873/.local/lib/python2.7/site-packages/pandas/__init__.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (hashtable as _hashtable,
/gpfs/fs01/user/s0f2-ba03446bdf62cc-bd5847e99873/.local/lib/python2.7/site-packages/pandas/core/dtypes/common.py:6: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import algos, lib
/gpfs/fs01/user/s0f2-ba03446bdf62cc-bd5847e99873/.local/lib/python2.7/site-packages/pandas/core/util/hashing.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 

Then we create an MLContext to interface with Apache SystemML. Note that we pass a SparkSession object as parameter so SystemML now knows how to talk to the Apache Spark cluster

In [3]:
ml = MLContext(spark)

Now we create some large random matrices to have numpy and SystemML crunch on it

In [4]:
u = np.random.rand(1000,10000)
s = np.random.rand(10000,1000)
w = np.random.rand(1000,1000)

Now we implement a short one-liner to define a very simple linear algebra operation

In case you are not familiar with matrix-matrix multiplication: https://en.wikipedia.org/wiki/Matrix_multiplication

sum(U' * (W . (U * S)))


| Legend        |            |   
| ------------- |-------------| 
| '      | transpose of a matrix | 
| * | matrix-matrix multiplication      |  
| . | scalar multiplication      |   



In [5]:
start = time.time()
res = np.sum(u.T.dot(w * u.dot(s)))
print time.time()-start

0.197705030441


As you can see this executes perfectly fine. Note that this is even a very efficient execution because numpy uses a C/C++ backend which is known for it's performance. But what happens if U, S or W get such big that the available main memory cannot cope with it? Let's give it a try:

In [6]:
'''
u = np.random.rand(10000,100000)
s = np.random.rand(100000,10000)
w = np.random.rand(10000,10000)
'''

In [12]:
u = np.random.rand(10000,1100000)
s = np.random.rand(1100000,10000)
w = np.random.rand(10000,10000)

MemoryError: 

After a short while you should see a memory error. This is because the operating system process was not able to allocate enough memory for storing the numpy array on the heap. Now it's time to re-implement the very same operations as DML in SystemML, and this is your task. Just replace all ###your_code_goes_here sections with proper code, please consider the following table which contains all DML syntax you need:

| Syntax        |            |   
| ------------- |-------------| 
| t(M)      | transpose of a matrix, where M is the matrix | 
| %*% | matrix-matrix multiplication      |  
| * | scalar multiplication      |   

## Task

In [13]:
#res = np.sum(u.T.dot(w * u.dot(s)))
#res = sum(###your_code_goes_here(U) %*% (W * (U ###your_code_goes_here S)))
script = """
res = sum( t(U) %*% (W * (U %*% S)))
"""

To get consistent results we switch from a random matrix initialization to something deterministic

In [14]:
u = np.arange(100000).reshape((100, 1000))
s = np.arange(100000).reshape((1000, 100))
w = np.arange(10000).reshape((100, 100))

In [15]:
prog = dml(script).input('U', u).input('S', s).input('W', w).output('res')
res = ml.execute(prog).get('res')
print res

SystemML Statistics:
Total execution time:		0.029 sec.
Number of executed Spark inst:	3.


1.25260525922e+28


If everything runs fine you should get *1.25260525922e+28* as result. Feel free to submit your DML script to the grader now!

### Submission

In [16]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py

--2018-07-24 11:26:32--  https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2029 (2.0K) [text/plain]
Saving to: ‘rklib.py’

100%[======================================>] 2,029       --.-K/s   in 0s      

2018-07-24 11:26:32 (15.6 MB/s) - ‘rklib.py’ saved [2029/2029]



In [17]:
from rklib import submit
key = "esRk7vn-Eeej-BLTuYzd0g"
part = "fUxc8"

email = "roman.kazinnik@gmail.com" #"###_YOUR_CODE_GOES_HERE_###"
secret = "FmeLQKjwkiWKgqC2" #"###_YOUR_CODE_GOES_HERE_###"
submit(email, secret, key, part, [part], script)

Something went wrong, please have a look at the reponse of the grader
-------------------------
{"errorCode":"78kj3h8h6"}
-------------------------
